<a href="https://colab.research.google.com/github/darrengao628/3DInfomax/blob/master/setfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install setfit
!apt install git-lfs
from huggingface_hub import notebook_login

notebook_login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 KB 683.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [4]:
dataset_id = "sst2"
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"

In [5]:
from datasets import load_dataset

dataset = load_dataset(dataset_id)
dataset

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset sst2 downloaded and prepared to /root/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

In [6]:
from setfit import sample_dataset

train_dataset = sample_dataset(dataset["train"])
train_dataset

Filter:   0%|          | 0/67349 [00:00<?, ? examples/s]

Filter:   0%|          | 0/67349 [00:00<?, ? examples/s]

Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 16
})

In [7]:
eval_dataset = dataset["validation"] 


In [8]:
eval_dataset

Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 872
})

In [9]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained(model_id)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [10]:
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitTrainer

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    num_iterations=20,
    column_mapping={"sentence": "text", "label": "label"},
)

In [11]:
trainer.train()


Applying column mapping to training dataset
***** Running training *****
  Num examples = 640
  Num epochs = 1
  Total optimization steps = 40
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

# Try setfit on the seq location dataset

In [6]:
import pandas as pd


In [7]:
df = pd.read_csv("/content/100_seq.csv")

In [8]:
def data_prep(seq_path):# Data prepation 
    import pandas as pd
    df=pd.read_csv(seq_path)
    seq=df["seq"].tolist()
    labels=df["labels"].tolist()
    return seq,labels

In [9]:
seq,labels=data_prep("/content/100_seq.csv")
len(labels),len(seq)

(100, 100)

In [10]:
from sklearn.model_selection import train_test_split

from datasets import Dataset
train_seq, test_seq, train_labels, test_labels = train_test_split(seq, labels, test_size =0.25,shuffle=True)
train_ds = Dataset.from_dict({"sentence": train_seq, "label": train_labels})
eval_ds = Dataset.from_dict({"sentence": test_seq, "label": test_labels})

In [12]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained(model_id)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [16]:
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitTrainer

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    loss_class=CosineSimilarityLoss,
    num_iterations=20,
    num_epochs=5,
    column_mapping={"sentence": "text", "label": "label"},
)

In [17]:
trainer.train()


Applying column mapping to training dataset
***** Running training *****
  Num examples = 3000
  Num epochs = 5
  Total optimization steps = 940
  Total train batch size = 16


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/188 [00:00<?, ?it/s]

Iteration:   0%|          | 0/188 [00:00<?, ?it/s]

Iteration:   0%|          | 0/188 [00:00<?, ?it/s]

Iteration:   0%|          | 0/188 [00:00<?, ?it/s]

Iteration:   0%|          | 0/188 [00:00<?, ?it/s]

In [18]:
metrics = trainer.evaluate()
metrics

Applying column mapping to evaluation dataset
***** Running evaluation *****


{'accuracy': 0.44}

# Full Code

In [22]:
import pandas as pd
def data_prep(seq_path):# Data prepation 
    import pandas as pd
    df=pd.read_csv(seq_path)
    seq=df["Sequence"].tolist()
    labels=df["labels"].tolist()
    return seq,labels



In [23]:
seq,labels=data_prep("/content/500_sequences.csv")

KeyError: ignored

In [19]:
import pandas as pd
def data_prep(seq_path):# Data prepation 
    import pandas as pd
    df=pd.read_csv(seq_path)
    seq=df["seq"].tolist()
    labels=df["labels"].tolist()
    return seq,labels


seq,labels=data_prep("/content/500_sequences.csv")

from sklearn.model_selection import train_test_split

from datasets import Dataset
train_seq, test_seq, train_labels, test_labels = train_test_split(seq, labels, test_size =0.25,shuffle=True)
train_ds = Dataset.from_dict({"sentence": train_seq, "label": train_labels})
eval_ds = Dataset.from_dict({"sentence": test_seq, "label": test_labels})

from setfit import SetFitModel

model = SetFitModel.from_pretrained(model_id)

from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitTrainer

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    loss_class=CosineSimilarityLoss,
    num_iterations=20,
    num_epochs=5,
    column_mapping={"sentence": "text", "label": "label"},
)

trainer.train()

metrics = trainer.evaluate()
metrics


KeyError: ignored